In [1]:
!cd /content && git clone https://github.com/ntua-el21050/bd3lms.git

Cloning into 'bd3lms'...
remote: Enumerating objects: 882, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 882 (delta 261), reused 248 (delta 205), pack-reused 542 (from 1)
Receiving objects: 100% (882/882), 3.13 MiB | 31.07 MiB/s, done.
Resolving deltas: 100% (553/553), done.


In [2]:
!ls -l /content/bd3lms

total 5040
-rw-r--r-- 1 root root  862037 Jan 26 23:57  2503.09573v3.pdf
drwxr-xr-x 9 root root    4096 Jan 26 23:57  configs
-rw-r--r-- 1 root root   33814 Jan 26 23:57  dataloader.py
-rw-r--r-- 1 root root   46696 Jan 26 23:57  diffusion.py
-rw-r--r-- 1 root root   56775 Jan 26 23:57  extension1_edm.ipynb
-rw-r--r-- 1 root root   58511 Jan 26 23:57  extension1_fm.ipynb
-rw-r--r-- 1 root root   58620 Jan 26 23:57  extension1_iddpm.ipynb
-rw-r--r-- 1 root root   58457 Jan 26 23:57  extension1_sigmoid.ipynb
-rw-r--r-- 1 root root   59098 Jan 26 23:57  extension1_sigmoid_val.ipynb
-rw-r--r-- 1 root root   58505 Jan 26 23:57  extension1_simple.ipynb
drwxr-xr-x 2 root root    4096 Jan 26 23:57  final_presentation
-rw-r--r-- 1 root root  225205 Jan 26 23:57  graphical_abstract.png
-rw-r--r-- 1 root root   11357 Jan 26 23:57  LICENSE
-rw-r--r-- 1 root root    7873 Jan 26 23:57  main.py
-rw-r--r-- 1 root root    8405 Jan 26 23:57  metrics.py
-rw-r--r-- 1 root root  257463 Jan 26 23:57 'midter

In [3]:
!pip install -q \
    torchmetrics==1.6.2 \
    datasets==3.3.2 \
    einops==0.8.1 \
    fsspec==2024.2.0 \
    hydra-core==1.3.2 \
    lightning==2.5.0.post0 \
    omegaconf==2.3.0 \
    packaging==23.2 \
    pandas==2.2.1 \
    rich==13.7.1 \
    scikit-learn==1.5.1 \
    timm==0.9.16 \
    transformers==4.49.0 \
    matplotlib==3.10.0 \
    wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 142.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Mount Google Drive (Colab) and choose where to save checkpoints
from pathlib import Path
import os

try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
    IN_COLAB = True
except Exception as e:
    IN_COLAB = False
    print("Not running in Colab (Drive not mounted):", e)

# Always save checkpoints to local runtime (Drive mount can break atomic saves).
SAVE_ROOT = Path("/content/repro_runs")
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

# Optional: sync copies of runs to Drive after training completes.
DRIVE_SAVE_ROOT = None
if IN_COLAB:
    DRIVE_SAVE_ROOT = Path("/content/drive/MyDrive/DIFF_2")
    DRIVE_SAVE_ROOT.mkdir(parents=True, exist_ok=True)

print("Local checkpoint save root:", SAVE_ROOT)
print("Drive sync root:", DRIVE_SAVE_ROOT)

Mounted at /content/drive
Local checkpoint save root: /content/repro_runs
Drive sync root: /content/drive/MyDrive/DIFF_2


In [5]:
# Helpers to run Hydra commands and parse val/ppl from output
import os
import sys
import shutil
import subprocess
import re
from pathlib import Path

# If a previous cell defined SAVE_ROOT/DRIVE_SAVE_ROOT, keep them; otherwise default to local runtime.
if "SAVE_ROOT" not in globals():
    SAVE_ROOT = Path(os.environ.get("BD3LMS_SAVE_ROOT", "/content/repro_runs"))
SAVE_ROOT = Path(SAVE_ROOT)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

if "DRIVE_SAVE_ROOT" not in globals():
    DRIVE_SAVE_ROOT = None
if DRIVE_SAVE_ROOT is not None:
    DRIVE_SAVE_ROOT = Path(DRIVE_SAVE_ROOT)
    DRIVE_SAVE_ROOT.mkdir(parents=True, exist_ok=True)

def _sync_tree(src: Path, dst: Path):
    """Best-effort incremental sync from local runtime to Drive."""
    src = Path(src)
    dst = Path(dst)
    for p in src.rglob("*"):
        rel = p.relative_to(src)
        q = dst / rel
        if p.is_dir():
            q.mkdir(parents=True, exist_ok=True)
            continue
        if p.is_file():
            q.parent.mkdir(parents=True, exist_ok=True)
            try:
                if q.exists() and q.stat().st_size == p.stat().st_size:
                    continue
            except OSError:
                pass
            shutil.copy2(p, q)

def sync_run_to_drive(run_dir: Path):
    """Sync a single run directory to Drive (if mounted)."""
    if DRIVE_SAVE_ROOT is None:
        return
    run_dir = Path(run_dir)
    dst = DRIVE_SAVE_ROOT / run_dir.name
    dst.mkdir(parents=True, exist_ok=True)
    _sync_tree(run_dir, dst)

def run_main(overrides, timeout=None):
    """Run `python -u main.py ...` and return combined stdout/stderr text."""
    env = dict(os.environ)
    env.setdefault("HYDRA_FULL_ERROR", "1")
    cmd = [sys.executable, "-u", "bd3lms/main.py", *overrides]
    print("\n$", " ".join(cmd))
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        timeout=timeout,
        check=False,
        env=env,
    )
    print(proc.stdout[-4000:])  # tail for quick visibility
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with return code {proc.returncode}")
    return proc.stdout

_METRIC_PATTERNS = [
    # Key: value (some loggers print this)
    re.compile(r"val/ppl\s*[:=]\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),
    re.compile(r"'val/ppl'\s*:\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),

    # Lightning "rich" table row (note the unicode box character │)
    re.compile(r"val/ppl\s*[│|]\s*([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", re.IGNORECASE),
]

def extract_val_ppl(log_text: str):
    # First try line-based parse from the end (most reliable for tables)
    for line in reversed(log_text.splitlines()):
        if "val/ppl" in line.lower():
            m = re.search(r"val/ppl.*?([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", line, re.IGNORECASE)
            if m:
                return float(m.group(1))

    # Fallback: scan entire text with known patterns
    hits = []
    for pat in _METRIC_PATTERNS:
        hits.extend(pat.findall(log_text))
    return float(hits[-1]) if hits else None

def _small_loader_overrides(batch_size=8, num_workers=2):
    """Overrides needed to avoid huge default batch sizes on Colab."""
    return [
        f"loader.global_batch_size={batch_size}",
        f"loader.eval_global_batch_size={batch_size}",
        f"loader.batch_size={batch_size}",
        f"loader.eval_batch_size={batch_size}",
        f"loader.num_workers={num_workers}",
        "trainer.accumulate_grad_batches=1",
    ]

def train_run(
    run_name,
    algo,
    block_size=None,
    from_pretrained=None,
    resume_ckpt_path=None,
    max_steps=500,
    extra_overrides=None,
    timeout=None,
    ):
    """
    Train a model and return the last.ckpt path.

    - If `resume_ckpt_path` is set, this uses Lightning resume (`checkpointing.resume_from_ckpt=true`)
      and continues training state (optimizer/scheduler/global_step) from that checkpoint.
    - Otherwise, if `from_pretrained` is set, this loads weights only (`training.from_pretrained=...`).

    Checkpoints are saved under SAVE_ROOT/<run_name>/checkpoints/last.ckpt.
    """
    save_dir = Path(SAVE_ROOT) / run_name
    if save_dir.exists():
        shutil.rmtree(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    overrides = [
        "mode=train",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        "data.max_train_samples=5000",
        # For LM1B, validation uses the 'test' split in this codebase
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "trainer.log_every_n_steps=10",
        "trainer.val_check_interval=50",
        f"trainer.max_steps={max_steps}",
        "data.max_valid_samples=500",
        "data.max_test_samples=500",
        f"checkpointing.save_dir={save_dir}",
        "wandb=null",
    ]
    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))
    if block_size is not None:
        overrides.append(f"block_size={block_size}")

    if resume_ckpt_path is not None:
        overrides.append("checkpointing.resume_from_ckpt=true")
        overrides.append(f"checkpointing.resume_ckpt_path={resume_ckpt_path}")
    else:
        overrides.append("checkpointing.resume_from_ckpt=false")
        if from_pretrained is not None:
            overrides.append(f"training.from_pretrained={from_pretrained}")

    if extra_overrides:
        overrides.extend(extra_overrides)

    _ = run_main(overrides, timeout=timeout)
    ckpt = save_dir / "checkpoints" / "last.ckpt"
    if not ckpt.exists():
        raise FileNotFoundError(f"Expected checkpoint not found: {ckpt}")

    # Best-effort sync to Drive (if available).
    sync_run_to_drive(save_dir)
    return str(ckpt)

def eval_run(algo, checkpoint_path, block_size=None, extra_overrides=None):
    """Evaluate perplexity (val/ppl) for a given checkpoint."""
    overrides = [
        "mode=ppl_eval",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        # For LM1B, `get_dataloaders` maps validation to the 'test' split
        "data.max_test_samples=1000",
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        f"eval.checkpoint_path={checkpoint_path}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "wandb=null",
        "data.max_valid_samples=100",
        "data.max_test_samples=100",
    ]

    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))
    if block_size is not None:
        overrides.append(f"block_size={block_size}")
    if extra_overrides:
        overrides.extend(extra_overrides)

    log_text = run_main(overrides)
    ppl = extract_val_ppl(log_text)
    if ppl is None:
        raise ValueError("Could not parse val/ppl from output. Try increasing the log tail or printing full logs.")
    return ppl

## 3) Run experiments and build a mini Table 3
This now mirrors the paper’s training procedure, but at tiny scale:
- Train a base BD3-LM at L'=L (here 128) once.
- Finetune that base for block sizes L' in {16, 8, 4} with noise-schedule-style resampling enabled.
- Also run AR, SEDD, and MDLM tiny baselines.
Only the scale (steps, samples, model size) is reduced for Colab feasibility.

In [6]:
from pathlib import Path
import os
import shutil
import sys
# import pandas as pd

results = []

# 1) Autoregressive baseline (tiny, scratch)
ar_run = "ar_tiny_len128"
ar_ckpt = train_run(ar_run, algo="ar")
ar_ppl = eval_run(algo="ar", checkpoint_path=ar_ckpt)
results.append({"model": "Autoregressive", "block_size_Lprime": "-", "val_ppl": ar_ppl})

# 2) Diffusion baselines (tiny, scratch): SEDD + MDLM
for algo_name, display_name in [("sedd", "SEDD"), ("mdlm", "MDLM")]:
    run_name = f"{algo_name}_tiny_len128"
    ckpt = train_run(
        run_name,
        algo=algo_name,
        extra_overrides=[
            "training.resample=false",
            "algo.var_min=false",
            "algo.clip_search_widths=[]",
        ],
    )
    ppl = eval_run(
        algo=algo_name,
        checkpoint_path=ckpt,
        extra_overrides=[
            "algo.var_min=false",
        ],
    )
    results.append({"model": display_name, "block_size_Lprime": "-", "val_ppl": ppl})

# 3) BD3LM methodology match (small-scale):
bd3lm_base_run = "bd3lm_base_len128"
bd3lm_base_ckpt = train_run(
    bd3lm_base_run,
    algo="bd3lm",
    block_size=128,
    extra_overrides=[
        "training.resample=false",
        "algo.var_min=false",
        "algo.clip_search_widths=[]",
        "training.sampling_eps_min=1e-3",
        "training.sampling_eps_max=0.5",
    ],
)

for Lprime in [16, 8, 4]:
    finetune_run = f"bd3lm_finetune_Lp{Lprime}"
    finetune_overrides = [
        "training.resample=true",
        "algo.var_min=false",
        "algo.clip_search_widths=[]",
    ]
    if Lprime == 16:
        finetune_overrides.extend([
            "training.sampling_eps_min=0.3",
            "training.sampling_eps_max=0.5",
        ])
    elif Lprime == 4:
        finetune_overrides.extend([
            "training.sampling_eps_min=0.5",
            "training.sampling_eps_max=1.0",
        ])
    finetune_ckpt = train_run(
        finetune_run,
        algo="bd3lm",
        block_size=Lprime,
        max_steps=600,
        resume_ckpt_path=bd3lm_base_ckpt,
        extra_overrides=finetune_overrides,
    )
    ppl = eval_run(
        algo="bd3lm",
        checkpoint_path=finetune_ckpt,
        block_size=Lprime,
        extra_overrides=[
            "algo.var_min=false",
        ],
    )
    results.append({"model": "Block diffusion (BD3LM)", "block_size_Lprime": Lprime, "val_ppl": ppl})


$ /usr/bin/python3 -u bd3lms/main.py mode=train data=lm1b-wrap data.cache_dir=/content/bd3lms/data data.streaming=true data.max_train_samples=5000 model=tiny model.length=128 model.attn_backend=sdpa algo=ar trainer.accelerator=gpu trainer.devices=1 trainer.num_nodes=1 trainer.precision=16-mixed trainer.num_sanity_val_steps=0 trainer.log_every_n_steps=10 trainer.val_check_interval=50 trainer.max_steps=500 data.max_valid_samples=500 data.max_test_samples=500 checkpointing.save_dir=/content/repro_runs/ar_tiny_len128 wandb=null loader.global_batch_size=8 loader.eval_global_batch_size=8 loader.batch_size=8 loader.eval_batch_size=8 loader.num_workers=2 trainer.accumulate_grad_batches=1 checkpointing.resume_from_ckpt=false
0% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
------------------------------------------------------------------------------------------

FileNotFoundError: Expected checkpoint not found: /content/repro_runs/bd3lm_finetune_Lp16/checkpoints/last.ckpt

In [ ]:
def print_table(rows):
    if not rows:
        print("No data to display.")
        return

    columns = list(rows[0].keys())

    str_rows = [
        {col: str(row.get(col, "")) for col in columns}
        for row in rows
    ]

    widths = {
        col: max(len(col), max(len(row[col]) for row in str_rows))
        for col in columns
    }

    def print_separator():
        print("+" + "+".join("-" * (widths[col] + 2) for col in columns) + "+")

    def print_row(row):
        print(
            "| " +
            " | ".join(row[col].ljust(widths[col]) for col in columns) +
            " |"
        )

    # Print table
    print_separator()
    print_row({col: col for col in columns})
    print_separator()
    for row in str_rows:
        print_row(row)
    print_separator()

print_table(results)